# Рекомендация тарифов

Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

**Цель исследования** - построить модель с максимально большим значением *accuracy*.

**Ход исследования**

Данные о поведении клиентов получим из файла `users_behavior.csv`. Предобработка данных не требуется.

Таким образом, исследование пройдет в 4 этапа:
1. Обзор данных;
2. Построение baseline моделей;
3. Улучшение моделей;
4. Прогноз на тестовой выборке.

## Обзор данных.

### Табличный осмотр.

Импортируем необходимые библиотеки.

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, MinMaxScaler

from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.dummy import DummyClassifier

Считаем данные из файла `users_behavior.csv`.

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')

Посмотрим на данные и получим общую информацию о данных с помощью метода `info()`.

In [3]:
df

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
...,...,...,...,...,...
3209,122.0,910.98,20.0,35124.90,1
3210,25.0,190.36,0.0,3275.61,0
3211,97.0,634.44,70.0,13974.06,0
3212,64.0,462.32,90.0,31239.78,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Посмотрим есть ли дубликаты строк.

In [5]:
df.duplicated().sum()

0

Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Из описания к данным:
* сalls — количество звонков,
* minutes — суммарная длительность звонков в минутах,
* messages — количество sms-сообщений,
* mb_used — израсходованный интернет-трафик в Мб,
* is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

**Вывод**

Наименование столбцов корректное. В данных отсутствуют пропуски и дубликаты.

## Построение baseline моделей.

В данном проекте стоит задача классификации. Будем использовать 3 модели:
1. Логистическая регрессия (LogisticRegression);
2. Дерево решений (DecisionTreeClassifier);
3. Случайный лес (RandomForestClassifier).

Посмотрим на распределение классов в данных.

In [6]:
df['is_ultra'].value_counts(normalize=True)

0    0.693528
1    0.306472
Name: is_ultra, dtype: float64

Разделим наши данные на 3 выборки - тренировочную, валидационную и тестовую. Данное преобразование будем осуществлять с помощью метода `train_test_split()` в 2 этапа:
1. Создадим выборку тренировочную и временную в соотношении 60:40;
2. Разделим временную выборку на 2 части в соотношении 50:50.

Итог, будем иметь 3 выборки - тренировочную, валидационную и тестовую в соотношении 60/20/20 соответственно.

Напишем функцию для данного преобразования.

In [7]:
def train_val_test_split(data_input, target_column, frac_train=0.6, frac_val=0.2, frac_test=0.2, random_state=17):
    
    X = data_input
    y = data_input[target_column]
    
    df_train, df_temp, y_train, y_temp = train_test_split(X, y, 
                                                          test_size = 1 - frac_train,
                                                          random_state=random_state)
    
    relative_test_size = frac_test / (frac_test + frac_val)
    
    df_val, df_test, y_val, y_test = train_test_split(df_temp, y_temp,
                                                      test_size=relative_test_size,
                                                      random_state=random_state)
    
    assert len(data_input) == len(df_train) + len(df_val) + len(df_test)
    
    return df_train, df_val, df_test

In [8]:
df_train, df_val, df_test = train_val_test_split(df, 'is_ultra')

Сделаем выборки непосредственно для моделей.

In [9]:
df_train, y_train = df_train.drop(['is_ultra'], axis=1), df_train['is_ultra']
df_val, y_val = df_val.drop(['is_ultra'], axis=1), df_val['is_ultra']
df_test, y_test = df_test.drop(['is_ultra'], axis=1), df_test['is_ultra']

Перейдем непосредственно к обучению. Сначала обучим модели без настройки параметров.

### Логистическая регрессия (LogisticRegression).

In [10]:
log_class = LogisticRegression(random_state=17)

log_class.fit(df_train, y_train)

y_pred = log_class.predict(df_val)

acc = accuracy_score(y_val, y_pred)

print(f'Доля верных ответов на валидации: {acc}')

Доля верных ответов на валидации: 0.76049766718507


### Дерево решений (DecisionTreeClassifier).

In [11]:
tree = DecisionTreeClassifier(random_state=17)

tree.fit(df_train, y_train)

y_pred = tree.predict(df_val)

acc = accuracy_score(y_val, y_pred)

print(f'Доля верных ответов на валидации: {acc}')

Доля верных ответов на валидации: 0.7107309486780715


### Случайный лес (RandomForestClassifier).

In [12]:
random_forest = RandomForestClassifier(random_state=17)

random_forest.fit(df_train, y_train)

y_pred = random_forest.predict(df_val)

acc = accuracy_score(y_val, y_pred)

print(f'Доля верных ответов на валидации: {acc}')

Доля верных ответов на валидации: 0.8009331259720062


**Вывод**

Без изменения каких-либо гиперпараметров, модель, построенная на алгоритме Случайного леса, показывает лучший результат на валидации (78%). Результат для логистической регрессии и дерева решений сопоставим и равен ~71-72%.

Данные результаты возьмем за основу и попробуем их улучшить с помощью настроек гиперпараметров.

## Улучшение моделей.

### Логистическая регрессия (LogisticRegression).

Для улучшения логистической регрессии попробуем 2 подхода:
1. Произведем трансформацию признаков с помощью `StandardScaler()` и `MinMaxScaler()`;
2. Создадим полиномиальные признаки с помощью `PolynomialFeatures()`.

#### Standart Scaling / MinMax Scaling

Создадим две выборки, в которых преобразуем данные с помощью

`StandardScaler()`

In [13]:
df_train_scaler = StandardScaler().fit_transform(df_train)

df_val_scaler = StandardScaler().fit_transform(df_val)

`MinMaxScaler()`

In [14]:
df_train_minmax = MinMaxScaler().fit_transform(df_train)

df_val_minmax = MinMaxScaler().fit_transform(df_val)

Для каждой выборки обучим логистическую регрессию.

`StandardScaler()`

In [15]:
log_class = LogisticRegression(random_state=17)

log_class.fit(df_train_scaler, y_train)

y_pred = log_class.predict(df_val_scaler)

acc = accuracy_score(y_val, y_pred)

print(f'Доля верных ответов: {acc}')

Доля верных ответов: 0.7636080870917574


`MinMaxScaler()`

In [16]:
log_class = LogisticRegression(random_state=17)

log_class.fit(df_train_minmax, y_train)

y_pred = log_class.predict(df_val_minmax)

acc = accuracy_score(y_val, y_pred)

print(f'Доля верных ответов: {acc}')

Доля верных ответов: 0.7418351477449455


#### PolynomialFeatures

Теперь для того, чтобы наша модель научилась строить нелинейную разделяющую поверхность, добавим новые полиномиальные признаки с помощью `PolynomialFeatures()` и найдем степень, которая дает лучший результат на валидационной выборке.

In [17]:
best_acc = 0.5
best_degree = None
best_train = None

for degree in range(1,10):
    poly = PolynomialFeatures(degree=degree)
    
    X_train_poly = poly.fit_transform(df_train_scaler)
    X_val_poly = poly.fit_transform(df_val_scaler)
    
    log_class = LogisticRegression(random_state=17)
    log_class.fit(X_train_poly, y_train)
    
    y_pred = log_class.predict(X_val_poly)
    y_pred_train = log_class.predict(X_train_poly)
    
    acc = accuracy_score(y_val, y_pred)
    acc_train = accuracy_score(y_train, y_pred_train)
    
    if acc > best_acc:
        best_acc = acc
        best_degree = degree
        best_train = acc_train


print(f'Доля верных ответов на тренировочной выборке равна {best_train}')
print(f'Доля верных ответов на валидации: {best_acc} и степенью полиномов {best_degree}')

Доля верных ответов на тренировочной выборке равна 0.8117219917012448
Доля верных ответов на валидации: 0.8133748055987559 и степенью полиномов 4


**Вывод**

С помощью преобразования исходных признаков и добавления новых полиномиальных признаков получилось увеличить долю верных ответов и довести до 81.3%

### Дерево решений (DecisionTreeClassifier).

Дерево решений имеет 3 основных гиперпараметра:
* `max_depth` - максимальная глубина дерева;
* `min_samples_split` - минимальное количество объектов, необходимое для разделения внутреннего узла;
* `min_samples_leaf` - минимальное число объектов в листе.

In [18]:
best_acc = 0.5
best_depth_dt = None
best_split_dt = None
best_leaf_dt = None
best_train = None

for depth in range(1,15):
    for split in range(2, 40, 3):
        for leaf in range(1, 20, 3):
            best_tree = DecisionTreeClassifier(max_depth=depth, min_samples_split=split,
                                               min_samples_leaf=leaf, random_state=17)
            best_tree.fit(df_train, y_train)
            
            y_pred = best_tree.predict(df_val)
            y_pred_train = best_tree.predict(df_train)
            
            acc = accuracy_score(y_val, y_pred)
            acc_train = accuracy_score(y_train, y_pred_train)
            
            if acc > best_acc:
                best_acc = acc
                best_depth_dt = depth
                best_split_dt = split
                best_leaf_dt = leaf
                best_train = acc_train

                
print(f'Доля верных ответов на тренировочной выборке равна {best_train}')
print(f'Доля верных ответов на валидации {best_acc} при глубине дерева {best_depth_dt},')
print(f'минимальным количеством объектов {best_split_dt}, числом объектов в листах {best_leaf_dt}')

Доля верных ответов на тренировочной выборке равна 0.8283195020746889
Доля верных ответов на валидации 0.8087091757387247 при глубине дерева 7,
минимальным количеством объектов 29, числом объектов в листах 4


**Вывод**

С помощью подбора гиперпараметров удалось улучшить результат на валидации на 9.8%

### Случайный лес (RandomForestClassifier).

Для случайного леса будем настраивать следующие гиперпараметры:
* `max_depth` - максимальная глубина дерева;
* `n_estimators` - количество деревьев в лесу;
* `min_samples_leaf` - минимальное число объектов в листе.

In [19]:
best_acc = 0.5
best_n_estimators = None
best_depth_rf = None
best_leaf_rf = None

for depth in range(1,15):
    for leaf in range(1, 15, 2):
        for est in range(1,102,10):
            best_random_forest = RandomForestClassifier(n_estimators=est, max_depth=depth,
                                                        min_samples_leaf=leaf, random_state=17)
            best_random_forest.fit(df_train, y_train)
            
            y_pred = best_random_forest.predict(df_val)
            y_pred_train = best_tree.predict(df_train)
            
            acc_train = accuracy_score(y_train, y_pred_train)
            acc = accuracy_score(y_val, y_pred)
            
            if acc > best_acc:
                best_acc = acc
                best_depth_rf = depth
                best_leaf_rf = leaf
                best_n_estimators=est
                

print(f'Доля верных ответов на тренировочной выборке: {best_train}')
print(f'Доля верных ответов на валидации: {best_acc} при глубине дерева {best_depth_rf} и')
print(f'числом объектов в листах {best_leaf_rf} с числом деревьев равным {best_n_estimators}')

Доля верных ответов на тренировочной выборке: 0.8283195020746889
Доля верных ответов на валидации: 0.8149300155520995 при глубине дерева 10 и
числом объектов в листах 1 с числом деревьев равным 51


**Вывод**

С помощью подбора гиперпараметров удалось улучшить результат на валидации на 3.4%.

## Проверка на тестовой выборке обученных моделей.

### Логистическая регрессия (LogisticRegression) + Standart Scaling + PolynomialFeatures

In [20]:
df_train_scaler = StandardScaler().fit_transform(df_train)

df_test_scaler = StandardScaler().fit_transform(df_test)

In [21]:
poly = PolynomialFeatures(degree=best_degree)

df_train_poly = poly.fit_transform(df_train_scaler)

df_test_poly = poly.fit_transform(df_test_scaler)

In [22]:
log_class = LogisticRegression(random_state=17)

log_class.fit(df_train_poly, y_train)

y_pred = log_class.predict(df_test_poly)

acc = accuracy_score(y_test, y_pred)

print(f'Доля верных ответов на тестовой выборке: {acc}')

Доля верных ответов на тестовой выборке: 0.8009331259720062


### Дерево решений (DecisionTreeClassifier).

In [23]:
best_tree = DecisionTreeClassifier(max_depth=best_depth_dt, min_samples_split=best_split_dt,
                                   min_samples_leaf=best_leaf_dt, random_state=17)

best_tree.fit(df_train, y_train)

y_pred = tree.predict(df_test)

acc = accuracy_score(y_test, y_pred)

print(f'Доля верных ответов на тестовой выборке: {acc}')

Доля верных ответов на тестовой выборке: 0.713841368584759


### Случайный лес (RandomForestClassifier).

In [24]:
best_random_forest = RandomForestClassifier(n_estimators=best_n_estimators, max_depth=best_depth_rf,
                                            min_samples_leaf=best_leaf_rf, random_state=17)

best_random_forest.fit(df_train, y_train)

y_pred = best_random_forest.predict(df_test)

acc = accuracy_score(y_test, y_pred)

print(f'Доля верных ответов на тестовой выборке: {acc}')

Доля верных ответов на тестовой выборке: 0.80248833592535


**Вывод**

По результатам на новых данных можно сказать следующее:
* Хуже всего себя показало Дерево решений. Результат на тестовой выборке - 71.3%, на валидации - 80.8%, что говорит о том, что дерево переобучилось и плохо справилось с новыми данными;
* Логистическая регрессия и Случайный лес показали схожие результаты (79,8% и 80.2% на тестовых данных соответственно);
* Для логистической регрессии потребовалось преобразовывать данные, а для Случайного леса потребовалась настройка гиперпараметров.

## Проверка моделей на адекватность.

Для проверки моделей на адекватность создадим `Dummy` классификатор из библиотеки `sklearn`, который просто предсказывает наиболее часто встречающейся класс.

In [25]:
dummy_clf = DummyClassifier(strategy="most_frequent", random_state=17)

dummy_clf.fit(df_train, y_train)

y_pred = dummy_clf.predict(df_test)

acc = accuracy_score(y_test, y_pred)

print(f'Доля верных ответов на тестовой выборке: {acc}')

Доля верных ответов на тестовой выборке: 0.6982892690513219


## Вывод.

В данном проекте были построены модели для задачи классификации, которые подбирают подходящий тариф.
Были рассмотрены 3 модели:
* Дерево решений;
* Логистическая регрессия;
* Случайный лес.

Наибольший прирост по метрике `accuracy` показала Логистическая регрессия (в baseline - 72%, тест - 79,8%). Но для этого потребовалось изменять исходные признаки, а так же добавлять новые признаки.

После настройки гиперпараметров оказалось, что Дерево решений переобучилось и плохо справилось с новыми данными (80.9% на валидации после настройки гиперпараметров и 71.3% на тестовой выборке).

Настройка гиперпараметров для Случайного леса не дала ощутимых результатов, как у Логистической регрессии (81.4% на валидации и 80.2% на тестовых данных). Данный классификатор оказался более устойчивым к переобучению, чем Дерево решений.

Так же был построен `Dummy` классификатор, который предсказывает наиболее часто встречающейся класс. Его результат на тестовой выборке оказался равен 69.8%. Доля верных ответов у всех моделей оказалась выше, чем у этого классификатора. Значит, что модели реально нашли какие-то зависимости в данных, а не просто *угадывали* ответ.